In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# first query: case data
query = """

SELECT
        crm.employee_key, crm.employee_name, call_chat.agent_path, skill_category, first_queue, last_queue, workday_id,bot_involved_flag,
        call_chat.case_key, call_chat.incidentlevel2reason, crm.sub_line_of_business__c, call_chat.wait_time, call_chat.complete_duration, call_chat.avg_response_cust, call_chat.avg_response_agnt, call_chat.agent_handle_time, call_chat.close_time,
        call_chat.no_of_upfront_sla_breach, call_chat.no_of_turns_per_conversation, call_chat.no_of_agents, call_chat.agent_tenure_days, Z.resolution_days, Z.customer_tenure_days,
        call_chat.csatscore, call_chat.cesscore, call_chat.agentsatisfactionscore, call_chat.fcrscore, call_chat.fiscal_qtr
    FROM
    (SELECT * FROM
    ce_analytics.fact_agg_call_chat where sales_support_flag = 'Support') call_chat
    join
    (select * from ce_analytics.dim_Crm_employee) crm
    on 
    crm.employee_key = call_chat.latest_employee_key
    
  
    JOIN
    (SELECT transaction_no, resolution_days, customer_tenure_days from ce_analytics.fact_closed_case) Z
    ON
    call_chat.case_key = Z.transaction_no
    WHERE
    call_chat.language_key = '13'
    AND
    call_chat.data_source = 'LE'
    
 

"""

In [ ]:
all_data = connection.read_sql(query, conn = conn)

In [ ]:
def check_string(string):
    lis = string.split('||')
    new = '||'.join(lis[:-1])
    return (new)

In [ ]:
# check_string('40BA0A16-52E1-E911-A870-000D3A3722D6||DYNAMIC_CRM||2021-04-22')  
all_data['new_employee_key'] = all_data.employee_key.map(check_string)

In [ ]:
# adding more agent data

query = """
select agent_case_handling_number.*, employee_key, Date, day_working_hours, day_billable_hour

from
( select latest_employee_key, date(date_parse(interaction_starttime,'%Y-%m-%d %H:%i:%s')) as day,  
    count(vendorid) as number_cases
    from ce_analytics.fact_agg_call_chat 
    group by latest_employee_key, date(date_parse(interaction_starttime,'%Y-%m-%d %H:%i:%s'))
    ) agent_case_handling_number

right join
(
select employee_key, date(date_parse(cal_date,'%Y-%m-%d %H:%i:%s')) as Date, sum(login_time)/3600.00 as day_working_hours, 
sum(billable_hour)/3600.00 as day_billable_hour 

from 
(select * from ce_analytics.fact_agent_productivity_combined where cal_date not like '%2022-04%' and 
cal_date not like '%2022-03%' and cal_date not like '%2022-05%' and cal_date not like '%2022-06%') new

group by employee_key, 
date(date_parse(cal_date,'%Y-%m-%d %H:%i:%s')) 
) agent_working_hours
on agent_case_handling_number.latest_employee_key = agent_working_hours.employee_key
and agent_case_handling_number.day = agent_working_hours.Date

"""

In [ ]:
agent_data = connection.read_sql(query, conn = conn)

In [ ]:
# fillna
import numpy as np
import pandas as pd
agent_data['latest_employee_key'] = agent_data['latest_employee_key'].fillna(agent_data['employee_key'])
# calculate CPH for agent level
agent_data['day'] = agent_data['day'].fillna(agent_data['Date'])
agent_data.number_cases = agent_data.number_cases.astype(float)
agent_data.day_billable_hour = agent_data.day_billable_hour.astype(float)
agent_data['CPH'] = agent_data.number_cases / agent_data.day_billable_hour 
# fill na
agent_data.replace([np.inf], np.nan, inplace=True) 
agent_data = agent_data[agent_data.CPH <= 10]

In [ ]:
agent_data.day_working_hours = agent_data.day_working_hours.astype(float)
agent_data.day_billable_hour = agent_data.day_billable_hour.astype(float)
agent_data.CPH = agent_data.CPH.astype(float)
agent_data['new_employee_key'] = agent_data.latest_employee_key.map(check_string)

agent = pd.DataFrame(agent_data.groupby('new_employee_key').agg({'day_working_hours':'median',
                                                             'day_billable_hour': 'median', 'CPH': np.nanmedian,
                                                                   'number_cases': 'sum'})).reset_index()

In [ ]:
# CHS data
query = """

select * from ragupt.fact_chs_scoring

"""
chs = connection.read_sql(query = query, conn = conn)
chs = chs.drop_duplicates()

#################################################################################### merge all the infomation
all_data_new = pd.merge(left = all_data, right = chs[['case_key', 'pred_score', 'model_prob']], on = 'case_key')
all_data_new.head()  # 2823621

case_number = pd.DataFrame(all_data_new.groupby(['employee_name','incidentlevel2reason']).case_key.count()).reset_index().rename(columns = {'case_key':'counting'})
all_data_new = pd.merge(left = all_data_new, right = case_number, on = ['employee_name', 'incidentlevel2reason'])
all_data_new = pd.merge(left = all_data_new, right = agent[['new_employee_key', 'day_working_hours', 'day_billable_hour', 'CPH', 'number_cases']]
                       ,left_on = 'new_employee_key', right_on = 'new_employee_key')

In [ ]:
########################################## build a new chs score
def new_chs(score):
    if score <= 0.35:
        return (0)
    if score > 0.35 and score < 0.7:
        return (1)
    
    if score >= 0.7:
        return (2)

all_data_new['sign'] = all_data_new.model_prob.map(new_chs)

all_data_new['sign'].value_counts()  # remove middle value
all_data_new_df = all_data_new[all_data_new.sign != 1]
all_data_new_df['pred_score'].value_counts() # check the distribution of the pred_score

In [ ]:
############################################################################################# modeling
import pandas as pd
from optbinning import Scorecard, BinningProcess
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load the train dataset
df_all_data_new = all_data_new_df[['employee_name', 'skill_category','first_queue', 'last_queue', 'bot_involved_flag',
                                   
                                   'incidentlevel2reason', 'sub_line_of_business__c',
       'wait_time', 'complete_duration', 'avg_response_cust',
       'avg_response_agnt', 'agent_handle_time', 'close_time',
       'no_of_upfront_sla_breach', 'no_of_turns_per_conversation',
       'no_of_agents', 'agent_tenure_days', 'resolution_days', 'customer_tenure_days', 'csatscore', 'day_working_hours',
       'day_billable_hour', 'CPH', 'chs', 'counting', 'number_cases']]
df_all_data_new.csatscore = df_all_data_new.csatscore.astype(float)
# df_all_data_new.agentsatisfactionscore = df_all_data_new.agentsatisfactionscore.astype(float)
# df_all_data_new.fcrscore = df_all_data_new.fcrscore.astype(float)
# df_all_data_new.cesscore = df_all_data_new.cesscore.astype(float)

In [ ]:
#################################################################################### check out train and test data
X = df_all_data_new.loc[:, df_all_data_new.columns != 'chs']
y = df_all_data_new['chs']

# Split the dataset into train and test
df_application_train, df_application_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

In [ ]:
logreg = LogisticRegression(C=5, max_iter=5000, random_state=161)
# from sklearn.ensemble import RandomForestClassifier
# randomforest = RandomForestClassifier(500)
# Define scaling method and values
import numpy as np
scaling_method = "min_max"
scaling_method_data = {"min": 0, "max": 100}
variable_names = np.array(X.columns)
# Instatiate and fit Scorecard
scorecard = Scorecard(
    binning_process= BinningProcess(variable_names),
    estimator=logreg,
    scaling_method="min_max",
     scaling_method_params={"min": 0, "max": 100},
    reverse_scorecard=True)

In [ ]:
scorecard.fit(df_application_train, list(y_train))

In [ ]:
# check model performance

from optbinning.scorecard.plots import plot_ks, plot_auc_roc
# y_test
# Assign score and predicted probability to test dataset
df_application_test.loc[:,"score"] = scorecard.score(df_application_test)

In [ ]:
# ROC-AUC plot
plot_auc_roc(y_test, df_application_test.score)  # done

In [ ]:
scorecard.table(style="detailed").to_excel('model_summary.xlsx')

In [ ]:
X['score'] = scorecard.score(X)
X['pred_score'] = y

final_score = pd.DataFrame(X.groupby(['employee_name','sub_line_of_business__c', 'incidentlevel2reason']).agg({'score':'mean', 'pred_score': 'count'})).reset_index()

final_score = final_score.rename(columns = {'pred_score': 'count'})
# final output
# final_score.pivot(index=['employee_name','sub_line_of_business__c'] ,columns='incidentlevel2reason',values=['score']).reset_index().to_excel('scorecard.xlsx')

In [ ]:
############################### ab testing
from scipy.stats import ttest_ind
import scipy.stats as stats


ab_test_list = ['wait_time',
       'complete_duration', 'avg_response_cust', 'avg_response_agnt',
       'agent_handle_time', 'close_time', 'no_of_upfront_sla_breach',
       'no_of_turns_per_conversation', 'no_of_agents', 'agent_tenure_days',
       'resolution_days', 'customer_tenure_days', 'csatscore', 
       'day_working_hours', 'day_billable_hour', 'CPH']

for check_attribute in ab_test_list:
    
    if check_attribute in ['no_of_agents', 'csatscore']:
        print ("======"+ check_attribute + " crosstab and ab test===========")
        print (pd.crosstab(all_data_new['pred_score'], all_data_new[check_attribute]))
        # multi proportion test
        print ('multi proportion test')
        print (stats.chi2_contingency(pd.crosstab(all_data_new['pred_score'], all_data_new[check_attribute])))
        
        
    else: 
        print ("======"+ check_attribute + " mean and ab test===========")
        print (all_data_new.groupby('pred_score')[check_attribute].mean())
        print (ttest_ind(
            all_data_new.dropna(subset = [check_attribute]).loc[all_data_new['pred_score'] == 0, check_attribute], 
            all_data_new.dropna(subset = [check_attribute]).loc[all_data_new['pred_score'] == 1, check_attribute],equal_var=True
        ) )

In [ ]:
import seaborn as sns
sns.displot(all_data_new, x="model_prob", bins=50)
sns.displot(all_data_new[all_data_new.pred_score == 0], x="model_prob", bins=50)  # < 0.2
sns.displot(all_data_new[all_data_new.csatscore.isna()], x="model_prob", bins=50)